In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy folders back into current Colab runtime
!cp -r /content/drive/MyDrive/landscapes_256 .
!cp -r /content/drive/MyDrive/outputs_baseline .
!cp -r /content/drive/MyDrive/outputs_swa .

Mounted at /content/drive


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model (ViT-B/32)
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").cuda()
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def extract_clip_features(image_folder, model, processor, batch_size=16):
    features = []

    image_paths = sorted([
        os.path.join(image_folder, f)
        for f in os.listdir(image_folder)
        if f.endswith(".png")
    ])

    for i in tqdm(range(0, len(image_paths), batch_size)):
        batch_paths = image_paths[i:i+batch_size]
        images = [Image.open(p).convert("RGB") for p in batch_paths]
        inputs = processor(images=images, return_tensors="pt", padding=True).to("cuda")

        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
            features.append(outputs.cpu())

    return torch.cat(features, dim=0)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
def compute_mmd(x, y, kernel='rbf', sigma=1.0):

    def rbf_kernel(a, b, sigma):
        dist = torch.cdist(a, b, p=2) ** 2
        return torch.exp(-dist / (2 * sigma ** 2))

    K_XX = rbf_kernel(x, x, sigma)
    K_YY = rbf_kernel(y, y, sigma)
    K_XY = rbf_kernel(x, y, sigma)

    m = x.size(0)
    n = y.size(0)

    mmd = K_XX.sum() / (m * m) + K_YY.sum() / (n * n) - 2 * K_XY.sum() / (m * n)
    return mmd.item()


In [ ]:
clip_real = extract_clip_features("landscapes_256", clip_model, clip_processor)
clip_base = extract_clip_features("outputs_baseline", clip_model, clip_processor)
clip_swa  = extract_clip_features("outputs_swa", clip_model, clip_processor)
cmmd_baseline = compute_mmd(clip_real, clip_base)
cmmd_swa = compute_mmd(clip_real, clip_swa)

print(f"CMMD (Baseline - Full Attention) = {cmmd_baseline:.4f}")
print(f"CMMD (SWA - Sliding Window) = {cmmd_swa:.4f}")


100%|██████████| 1/1 [00:00<00:00, 16.07it/s]

CMMD (Baseline - Full Attention) = 0.4544
CMMD (SWA - Sliding Window) = 0.4552


In [ ]:
import os

print("📁 Available folders:")
for f in os.listdir():
    if os.path.isdir(f):
        print(f"➡️ {f}")


📁 Available folders:
➡️ .config
➡️ outputs_swa
➡️ drive
➡️ outputs_baseline
➡️ landscapes_256
➡️ sample_data


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
from PIL import Image
import os
#kinda similar to 2b ke first few lines of code
os.makedirs("landscapes_256", exist_ok=True)
for i in range(5):
    img = Image.new('RGB', (256, 256), color=(100+i*20, 150, 200))
    img.save(f"landscapes_256/sample_{i}.png")


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload raw landscape .zip file


Saving kaggle.json to kaggle.json


In [ ]:
!unzip -q landscapes_256.zip


unzip:  cannot find or open landscapes_256.zip, landscapes_256.zip.zip or landscapes_256.zip.ZIP.


In [ ]:
import os
print(os.listdir())


['.config', 'outputs_swa', 'drive', 'outputs_baseline', 'kaggle.json', 'landscapes_256', 'sample_data']


In [ ]:
from transformers import AutoProcessor, AutoModel
import torch

# Load SigLIP and ALIGN from HuggingFace
siglip_model = AutoModel.from_pretrained("google/siglip-base-patch16-224").to("cuda")
siglip_processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-224")

align_model = AutoModel.from_pretrained("kakaobrain/align-base").to("cuda")
align_processor = AutoProcessor.from_pretrained("kakaobrain/align-base")


config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/690M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
siglip_real = extract_clip_features("landscapes_256", siglip_model, siglip_processor)
siglip_base = extract_clip_features("outputs_baseline", siglip_model, siglip_processor)
siglip_swa  = extract_clip_features("outputs_swa", siglip_model, siglip_processor)
align_real = extract_clip_features("landscapes_256", align_model, align_processor)
align_base = extract_clip_features("outputs_baseline", align_model, align_processor)
align_swa  = extract_clip_features("outputs_swa", align_model, align_processor)

100%|██████████| 1/1 [00:00<00:00,  9.46it/s]


In [ ]:
cmmd_siglip_base = compute_mmd(siglip_real, siglip_base)
cmmd_siglip_swa  = compute_mmd(siglip_real, siglip_swa)
cmmd_align_base = compute_mmd(align_real, align_base)
cmmd_align_swa  = compute_mmd(align_real, align_swa)
print(f"SigLIP CMMD (Baseline – Full Attention) = {cmmd_siglip_base:.4f}")
print(f"SigLIP CMMD (SWA – Sliding Window)     = {cmmd_siglip_swa:.4f}")
print(f"ALIGN CMMD (Baseline – Full Attention) = {cmmd_align_base:.4f}")
print(f"ALIGN CMMD (SWA – Sliding Window)     = {cmmd_align_swa:.4f}")


SigLIP CMMD (Baseline – Full Attention) = 0.4634
SigLIP CMMD (SWA – Sliding Window)     = 0.4634
ALIGN CMMD (Baseline – Full Attention) = 0.6323
ALIGN CMMD (SWA – Sliding Window)     = 0.6325
